# おためしWordBatch
---
メルカリコンペの時はpipでいけなかったような記憶が

**結論**  
sklearn.feature_extraction.text.HashingVectorizerとかのwrapper．  
メルカリコンペのデータでちょっと試した感じだと，sklearnと違ってマルチコア全部使ってくれるっぽい．  
メルカリコンペで流行ってたのは計算時間の問題だと思う．多分．

### Install
pip install wordbatchするだけ  
要Cython  
https://github.com/anttttti/Wordbatch

### Get started

In [7]:
from sklearn.feature_extraction.text import HashingVectorizer
#from sklearn.linear_model import *
#vct= HashingVectorizer()
#clf= SGDRegressor()

import wordbatch
from wordbatch.models import FTRL
from wordbatch.extractors import WordBag, WordHash, WordSeq, WordVec
wb= wordbatch.WordBatch(extractor=(WordBag, {"hash_ngrams":2, "hash_ngrams_weights":[0.5, -1.0], "hash_size":2**23, "norm":'l2', "tf":'log', "idf":50.0}))
# clf = FTRL(alpha=1.0, beta=1.0, L1=0.00001, L2=1.0, D=2 ** 25, iters=1)
clf = FTRL()
train_texts = ["Cut down a tree with a herring? It can't be done.",
              "Don't say that word.",
              "How can we not say the word if you don't tell us what it is?"]
train_labels = [1, 0, 1]
test_texts = ["Wait! I said it! I said it! Ooh! I said it again!"]

clf.fit(wb.transform(train_texts), train_labels)
preds= clf.predict(wb.transform(test_texts))
print(preds)

Normalize text
Extract wordbags
Normalize text
Extract wordbags
[0.50758516]


In [5]:
a = wb.transform(train_texts).toarray()
b = wb.transform(test_texts).toarray()
print(a.shape, b.shape)


Normalize text
Extract wordbags
Normalize text
Extract wordbags
(3, 8388608) (1, 8388608)


### 4種類のFeature Extracter

- **WordHash**  
is simply the Scikit-learn HashingVectorizer wrapped with the Wordbatch parallelization, providing multiplied processing speeds
- **WordBag**  
is a flexible alternative to Wordhash, providing cababilities missing from Scikit-learn, such as IDF and per n-gram order weighting of hashed features, windowed and distance-weighted polynomial interactions, and more transforms for counts.  
使うならこれ？WordHashだとn_gramを適用できないのが辛い．  
- **WordSeq**  
provides sequences of word integers, as used by the deep learning toolkits for input into LSTM models.  
kerasのtokenizerとどう違う？  
- **WordVec**  
provides embedding transforms from words into wordvectors  
予め単語ベクトルを与える必要があるっぽい．それを適用するだけ．

In [13]:
txt1 = ["Hello", "world", "Goodbye"]
txt2 = ["Hello world", "Goodbye world"]
params_wb = {"hash_ngrams":2, 
             "hash_ngrams_weights":[0.5, -1.0], 
             "hash_size":2**23, 
             "norm":'l2', 
             "tf":'log', 
             "idf":50.0}
wb = wordbatch.WordBatch(extractor=(WordBag, params_wb))
print(wb.transform(txt1))
print(wb.transform(txt2))

Normalize text
Extract wordbags
  (0, 784967)	0.9999999866502418
  (1, 6931205)	-0.9999999866502418
  (2, 6188862)	0.9999999866502418
Normalize text
Extract wordbags
  (0, 784967)	0.41252769709377807
  (0, 1216271)	0.8121833286211833
  (0, 6931205)	-0.41252769709377807
  (1, 2052107)	-0.8121833286211833
  (1, 6188862)	0.41252769709377807
  (1, 6931205)	-0.41252769709377807


In [12]:
txt1 = ["Hello", "world", "Goodbye"]
txt2 = ["Hello world", "Goodbye world"]
params_wb = {"norm":'l2'}
wb = wordbatch.WordBatch(extractor=(WordHash, params_wb))
print(wb.transform(txt1))
print(wb.transform(txt2))

Normalize text
Extract wordhashes
  (0, 784967)	1.0
  (1, 639749)	-1.0
  (2, 945982)	1.0
Normalize text
Extract wordhashes
  (0, 639749)	-0.7071067811865475
  (0, 784967)	0.7071067811865475
  (1, 639749)	-0.7071067811865475
  (1, 945982)	0.7071067811865475


In [11]:
txt1 = ["Hello", "world", "Goodbye", "Hello"]
txt2 = ["Hello world", "Goodbye world"]
params_wb = {"norm":'l1'}
wb = wordbatch.WordBatch(extractor=(WordSeq, params_wb))
print(wb.transform(txt1))
print(wb.transform(txt2))

Normalize text
Extract wordseqs
[[1], [2], [3], [1]]
Normalize text
Extract wordseqs
[[1, 2], [3, 2]]


In [10]:
txt1 = ["Hello", "world", "Goodbye", "Hello"]
txt2 = ["Hello world", "Goodbye world"]
params_hv = {"norm":'l1'}

vct = HashingVectorizer(params_hv)
print(vct.fit_transform(txt1))
print(vct.fit_transform(txt2))

  (0, 784967)	1.0
  (1, 639749)	-1.0
  (2, 945982)	1.0
  (3, 784967)	1.0
  (0, 639749)	-0.7071067811865475
  (0, 784967)	0.7071067811865475
  (1, 639749)	-0.7071067811865475
  (1, 945982)	0.7071067811865475
